In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/t5-base-lm-adapt")

In [2]:
from datasets import load_from_disk

cocktails_datasets = load_from_disk("cocktails_datasets")

In [3]:
def preprocess_function(examples):
    inputs = examples["inputs"]
    targets = examples["targets"]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [4]:
tokenized_datasets = cocktails_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=cocktails_datasets["train"].column_names,
)

Loading cached processed dataset at cocktails_datasets/train\cache-bc78a3a77a97337b.arrow
Loading cached processed dataset at cocktails_datasets/test\cache-65e0809fc2661f47.arrow


In [5]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-base")

In [6]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [7]:
import numpy as np
from datasets import load_metric

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [8]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"t5-small_coktails_recipe-base",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)

In [9]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1392
  Batch size = 2
100%|██████████| 696/696 [06:48<00:00,  1.70it/s]


{'eval_loss': 2.5310001373291016,
 'eval_bleu': 0.31022946988308936,
 'eval_runtime': 412.7092,
 'eval_samples_per_second': 3.373,
 'eval_steps_per_second': 1.686}

In [10]:
trainer.train()

***** Running training *****
  Num examples = 5564
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 16692
  0%|          | 0/16692 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.54 GiB already allocated; 15.99 MiB free; 2.77 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1392
  Batch size = 4
100%|██████████| 348/348 [02:29<00:00,  2.33it/s]


{'eval_loss': 1.1013939380645752,
 'eval_bleu': 7.2818226389977125,
 'eval_runtime': 150.2277,
 'eval_samples_per_second': 9.266,
 'eval_steps_per_second': 2.316,
 'epoch': 3.0}

In [39]:
from transformers import pipeline

model_checkpoint = "t5-small_coktails_recipe/checkpoint-8346"
translator = pipeline("text2text-generation", model=model_checkpoint)
translator('Bacardi Carta Blanca light rum,fresh Small ripe strawberries,Honey',  max_length=600)

loading configuration file t5-small_coktails_recipe/checkpoint-8346\config.json
Model config T5Config {
  "_name_or_path": "t5-small_coktails_recipe/checkpoint-8346",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vocab_size": 32128
}

loading configuration file t5-small_coktails_recipe/checkpoint-8346\config.json
Model config T5Config {
  "_name_or_path": "t5-small_coktails_recipe/checkpoint-8346",
  "architectures": [
    "T5ForConditionalGen

[{'generated_text': 'SHAKE all ingredients with ice and fine strain into chilled glass. 6 cl Bacardi Carta Blanca light rum 1.5 cl Fresh Small ripe strawberries 1.5 cl Fresh Small ripe strawberries 1.5 cl Honey'}]

In [22]:
cocktails_datasets["train"][0]

{'inputs': 'Bacardi Carta Blanca light rum,Dutch Cacao white crème de cacao,Del Maguey VIDA mezcal,Lime cordial',
 'targets': 'STIR all ingredients with ice and strain into chilled glass.\n4.5 cl Bacardi Carta Blanca light rum\n1 cl Dutch Cacao white crème de cacao\n0.5 cl Del Maguey VIDA mezcal\n2.5 cl Lime cordial'}

In [35]:
cocktails_datasets["train"][2]

{'inputs': 'Small ripe strawberries,Ketel One Vodka,Lemon juice,Sugar syrup,Thomas Henry Soda Water',
 'targets': 'MUDDLE strawberries in base of shaker. Add next three ingredients, SHAKE with ice and fine strain into ice-filled glass. TOP with soda and serve with straws.\n2 fresh Small ripe strawberries\n6 cl Ketel One Vodka\n4.5 cl Lemon juice\n2.25 cl Sugar syrup\nTop up with Thomas Henry Soda Water'}